In [1]:
Packages <- c("dplyr","broom", "data.table",  "ggplot2", "bbmle", "dplyr","tidyr", "stringr", "tidyverse", "lubridate", "RColorBrewer")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")

#clownfish metadata
load("~/parentage/r_data/total_sampling_across_years.RData")
load("~/parentage/r_data/sampled_area_each_year.RData")
#https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Script_outputs/cumulative_prop_hab_sampled_by_site.RData)
load("~/parentage/r_data/cumulative_prop_hab_sampled_by_site.RData")
#download.file(url = "https://github.com/pinskylab/genomics/blob/master/data/fish-obs.RData?raw=true", destfile = "~/parentage/r_data/fish-obs.RData")
fish_obs <- readRDS("~/parentage/r_data/fish-obs.RData") 
load("~/parentage/r_data/site_dist_info.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/anem_db.RData?raw=true", destfile = "~/parentage/r_data/anem_db.RData")
load("~/parentage/r_data/anem_db.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/dives_db.RData?raw=true", destfile = "~/parentage/r_data/dives_db.RData")
load("~/parentage/r_data/dives_db.RData")
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Data_from_database/fish_db.RData?raw=true", destfile = "~/parentage/r_data/dives_db.RData")
load("~/parentage/r_data/fish_db.RData")
load("~/parentage/r_data/gps_db.RData")

In [2]:
UnsampledSites <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "Pangasugan", "Other") 
SimConn <- read.csv(file="~/oceanography/script_output/SimConnectivityTableWithMeta.csv", header=T) #%>%
    #filter(source %!in% UnsampledSites & destination  %!in% UnsampledSites )

In [10]:
#read in the df where each particle is an observation
Simulation2012 <- fread(file="~/oceanography/script_output/SimConnLongForm2011SamplingYear.csv", showProgress = T)
#this is about 3 GB so fread is fastest, but consider these options for dealing with a large csv
#https://inbo.github.io/tutorials/tutorials/r_large_data_files_handling/

#head(Simulation2012 %>% group_by(date, source, destination) %>% summarise(Check=n()))
#CAI should be 201718, it is, all good

In [24]:
#proportion sampled

#add how well we sampled sites- proportion of habitat sampled
load("~/parentage/r_data/cumulative_prop_hab_sampled_by_site.RData")
PropSamp <- cumulative_prop_hab_sampled_by_site %>%
    mutate(total_possible_sample_anems = ifelse(site=="Caridad Proper", 4, total_possible_sample_anems) ) %>%
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(site=="Caridad Proper" & total_anems_sampled==4, 1, total_prop_hab_sampled_anems_tidied) ) %>%
    mutate(total_possible_sample_anems = ifelse(site=="Sitio Lonas", total_anems_sampled, total_possible_sample_anems) ) %>%
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(site=="Sitio Lonas", 1, total_prop_hab_sampled_anems_tidied) )%>%
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(is.nan(total_prop_hab_sampled_anems_tidied), 0, total_prop_hab_sampled_anems_tidied) ) %>%
    dplyr::select(site, time_frame, end_year, total_prop_hab_sampled_anems_tidied) 

PropSamp$site <- gsub(". ", ".", PropSamp$site, fixed=TRUE) #fix spaces in Magbangon names

S.Mag <- PropSamp %>%#make a table for all of the S. Mabangon years/prop sampled. Then join to the DF, make a column adding S.Magbangon values to all rows of prop sampled, but only sub that value in for PropSamp in N.Magbangon rows, then rename N.Magbangon as Magbangon
    filter(site=="S.Magbangon") %>%
    dplyr::select(site, end_year, total_prop_hab_sampled_anems_tidied) %>%
    rename(S.MagVal="total_prop_hab_sampled_anems_tidied", extra="site")

PropSamp <- left_join(PropSamp, S.Mag, by=c("end_year")) %>%
    mutate(S.MagSum=total_prop_hab_sampled_anems_tidied+S.MagVal) %>% #create col adding the S.Mag values to prop hab - keep in mind values could be greater than 1- if so change them to 1
    mutate(total_prop_hab_sampled_anems_tidied= ifelse(site=="N.Magbangon", S.MagSum, total_prop_hab_sampled_anems_tidied)) %>%#sub this value in for only N.Mag
    mutate(site=ifelse(site=="N.Magbangon", "Magbangon", site)) %>%#change N.Mag name to generic Mag
    filter(site !="S.Magbangon") %>%#eliminate the S.Mag rows, they are now repeats
    dplyr::select(-S.MagVal,-extra, -S.MagSum, -time_frame) %>%
    rename(PropSamp="total_prop_hab_sampled_anems_tidied")


#check if there are values>1, should be none
sum(which(PropSamp$total_prop_hab_sampled_anems_tidied >1)) #zero, that's good

[1] 0

In [11]:
#load data from the genetic sampling in each year
#read in the kernel fitting summary
kernels <- read.csv(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv", header=T)

##read in the sites that we sampled each year
N_gen_par <- read.table(file="~/parentage/colony2/20190523_1340loci/input/all_parents_corrected.txt", header = TRUE, stringsAsFactors = F) %>%#not sure that I need the parents here
    mutate(fish_indiv=as.character(fish_indiv))
N_gen_offs <- read.table(file="~/parentage/colony2/20190523_1340loci/input/all_offspring_corrected.txt", header=T, stringsAsFactors = F) %>%
    mutate(fish_indiv=as.character(fish_indiv))



#gather the summary of total offspring sampled
#from Allison, putting all the meta data together (Constants_database_common_functions.R)
##### Match up other relevant info (site, date, fish_indiv, etc.) to fish in the clownfish table
# Pull out year and month into a separate column in dives_db
dives_db_processed <- dives_db %>%
  mutate(year = as.integer(substring(date,1,4))) %>%
  mutate(month = as.integer(substring(date,6,7))) %>%
  mutate(dive_date = date(date))

# Pull all APCL caught or otherwise in the clownfish table
allfish_fish <- fish_db %>%
  select(fish_table_id, anem_table_id, fish_spp, sample_id, anem_table_id, recap, tag_id, color, sex, size, fish_obs_time, fish_notes) %>%
  filter(fish_spp == 'APCL') %>%
  mutate(size = as.numeric(size))  # make the size numeric (rather than chr) so can do means and such

# and their corresponding anemones
allfish_anems <- anem_db %>%
  select(anem_table_id, dive_table_id, anem_obs, anem_id, old_anem_id, anem_notes) %>%
  filter(anem_table_id %in% allfish_fish$anem_table_id)

# and the corresponding dive info
allfish_dives <- dives_db_processed %>%
  select(dive_table_id, dive_type, date, year, month, site, gps, dive_notes) %>%
  filter(dive_table_id %in% allfish_anems$dive_table_id) 

# join together
allfish_caught <- left_join(allfish_fish, allfish_anems, by="anem_table_id")
allfish_caught <- left_join(allfish_caught, allfish_dives, by="dive_table_id")

# add in the gen_ids and fish_indiv (now in a separate gen_id table) - gen_id only comes in the time the fish was sequenced, not at all captures
allfish_caught <- left_join(allfish_caught, (fish_obs %>% select(fish_table_id, gen_id, fish_indiv)), by = "fish_table_id") %>%
    select(fish_indiv, sample_id, site) %>%
    mutate()

N_gen_offs_annual  <- left_join(N_gen_offs, allfish_caught, by=c("fish_indiv", "sample_id")) %>% 
    group_by(year, site) %>%
    summarise(n_offs_gen=n()) %>%
    ungroup()

N_gen_offs_annual$site <- gsub(". ", ".", N_gen_offs_annual$site, fixed=TRUE)

##for all years
N_gen_offs_all <- N_gen_offs_annual %>% 
    group_by(site) %>% 
    summarise(sampled_fish=sum(n_offs_gen, na.rm=T))

sum(N_gen_offs_all$sampled_fish)

`summarise()` regrouping output by 'year' (override with `.groups` argument)
`summarise()` ungrouping output (override with `.groups` argument)


[1] 791

In [12]:
#combine N/S Magbangon in the genetic sampling data

AnnualRecsSamp <- bind_rows(N_gen_offs_annual %>%
                        filter(site !="N.Magbangon" & site!="S.Magbangon"),
                    N_gen_offs_annual %>%
                        mutate(Magbangon=ifelse(site=="N.Magbangon" | site=="S.Magbangon", "Magbangon", "no")) %>%
                        filter(Magbangon=="Magbangon") %>%
                        group_by(year, Magbangon) %>%
                        summarise(sum_offs=sum(n_offs_gen)) %>%
                        rename(site="Magbangon", n_offs_gen="sum_offs"))
sum(AnnualRecsSamp$n_offs_gen) #should be 791

`summarise()` regrouping output by 'year' (override with `.groups` argument)


[1] 791

In [17]:
#sample the simulation results according to how many offspring we sampled at that site in real life, each row is a column
NGenOffs2012 <- AnnualRecsSamp %>%
    filter(year=="2012")

#make an empty dataframe
SimulatedSampling2012 = Simulation2012[FALSE,]

for(i in 1:nrow(NGenOffs2012)){ 
    
    destination_eval <- as.character(NGenOffs2012$site[i])
    
    SimSampDestination <- Simulation2012 %>%
                                filter(destination==destination_eval) %>%
                                sample_n(as.numeric(NGenOffs2012$n_offs_gen[i]), replace=F)
    SimulatedSampling2012 <- bind_rows(SimulatedSampling2012, SimSampDestination)
}

nrow(SimulatedSampling2012)==sum(NGenOffs2012$n_offs_gen) #should be TRUE

#now randomly assign parentage match status to some of these rows
NumPar <- as.numeric(kernels %>%
    filter(Year=="2012") %>%
    select(NumParentageMatches))

SimulatedSampling2012Par <- SimulatedSampling2012 %>%
    sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
    mutate(Parentage=1) %>% #assign these rows a positive match value
    select(date, source, destination, Parentage)#drop columns that will result in repeats after a join

#join the parentage match df with the full df, adding the parentage column to sort the connectivity matrix by
SimulatedSampling2012Par <- left_join(SimulatedSampling2012, SimulatedSampling2012Par, by=c("date", "source", "destination")) %>%
    
nrow(SimulatedSampling2012Par) #for 2012 it should be 63, and it is yay
nrow(SimulatedSampling2012Par %>% filter(Parentage==1)) #for 2012 should be 3, and it is yay

[1] TRUE

[1] 63

[1] 3

In [ ]:
#shoot... I have to figure out how to add in rows/cols for all of the sites that we sampled but didn't find fish at, need to figure out at which point this needs to happen in the code after lab meeting

In [27]:
head(PropSamp %>% filter(end_year =="2012" & PropSamp >0))

site,end_year,PropSamp
Cabatoan,2012,0.4230769
Magbangon,2012,0.4593838
Palanas,2012,0.2898551
Poroc Rose,2012,1.0000000
Poroc San Flower,2012,1.0000000
San Agustin,2012,0.8888889


In [14]:
head(SimulatedSampling2012)
head(SimulatedSampling2012Par)

date,source,destination,SourceSampled,DestSampled,SimMonth,SimDay,SimYear,YearSampled,SimMonsoon,DailyParticles,ParticlesReleasedDaily,dist_km,bearing,direction
2011-02-10,Hicgop South,Cabatoan,yes,yes,2,2,2011,2011,NEM,51,1984,6.114777,-40.86575,319.1343
2011-03-26,Sitio Baybayon,Cabatoan,yes,yes,3,3,2011,2011,NEM,24,1984,24.910562,-12.35333,347.6467
2011-05-03,SF1,Cabatoan,no,yes,5,5,2011,2011,NEM,11,1984,1.467208,123.43161,123.4316
2010-12-19,SF1,Cabatoan,no,yes,12,12,2010,2011,NEM,415,1984,1.467208,123.43161,123.4316
2011-03-25,Sitio Baybayon,Palanas,yes,yes,3,3,2011,2011,NEM,58,1984,27.291943,-14.97711,345.0229
2011-04-27,Sitio Baybayon,Palanas,yes,yes,4,4,2011,2011,NEM,30,1984,27.291943,-14.97711,345.0229


date,source,destination,SourceSampled,DestSampled,SimMonth,SimDay,SimYear,YearSampled,SimMonsoon,DailyParticles,ParticlesReleasedDaily,dist_km,bearing,direction,Parentage
2011-02-10,Hicgop South,Cabatoan,yes,yes,2,2,2011,2011,NEM,51,1984,6.114777,-40.86575,319.1343,no
2011-03-26,Sitio Baybayon,Cabatoan,yes,yes,3,3,2011,2011,NEM,24,1984,24.910562,-12.35333,347.6467,no
2011-05-03,SF1,Cabatoan,no,yes,5,5,2011,2011,NEM,11,1984,1.467208,123.43161,123.4316,no
2010-12-19,SF1,Cabatoan,no,yes,12,12,2010,2011,NEM,415,1984,1.467208,123.43161,123.4316,no
2011-03-25,Sitio Baybayon,Palanas,yes,yes,3,3,2011,2011,NEM,58,1984,27.291943,-14.97711,345.0229,no
2011-04-27,Sitio Baybayon,Palanas,yes,yes,4,4,2011,2011,NEM,30,1984,27.291943,-14.97711,345.0229,no


In [19]:
#NOTE going to need to make sure everything with site names is in alphabetical order!
SimDisp2012Mat <- SimulatedSampling2012Par %>%
    group_by(destination, source) %>%
    summarise(NumPar=sum(Parentage)) %>%
    arrange(destination, source) %>%
    spread(destination, NumPar)
#
rownames(SimDisp2012Mat) <- SimDisp2012Mat$source
SimDisp2012Mat$source <- NULL

`summarise()` regrouping output by 'destination' (override with `.groups` argument)
Warning message:
“Setting row names on a tibble is deprecated.”

In [23]:
nrow(SimDisp2012Mat)
ncol(SimDisp2012Mat)

[1] 15

[1] 8

In [ ]:
#add in unassigned 
RowtoAddInt <- SimulatedSampling2012Par %>%
    ungroup() %>%
    arrange(destination) %>%
    dplyr::select(TotalUnassigned)

RowtoAdd <- as.data.frame(t(RowtoAddInt))
colnames(RowtoAdd) <- colnames(SimDisp2013Mat)

FullSimDisp2013 <- rbind(SimDisp2013Mat, RowtoAdd)
rownames(FullSimDisp2013) <- NULL

In [ ]:
FullSimDisp2012

In [ ]:
SimDisp2012Mat

In [ ]:
#turn into parentage matrix

#SimDisp2012Mat <- SimDisp2012 %>%
#    ungroup() %>%
#    spread(source, destination, SumParticles)
    

#make the rest of the kernel input files
Assignments <- FullSimDisp2013
Adult_sample_proportions <- matrix(nrow=18, ncol=1, 1) #we sampled all sites completely because we are working with simulated data
Sampled_reefs <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/site_index_all.csv", header=FALSE))[,18] #sampled all reefs in the distance matrix/area/centroid files
Distances <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/distance_matrix_unsurveyed.csv", header=FALSE))[-2, -2]  #drop a magbangon row  
Reef_sizes <- as.matrix(read.csv("~/parentage/kernel_fitting/894_loci/area_unsurveyed.csv", header=FALSE))[]
Centroids <- as.matrix(read.csv("~/parentage/kernel_fitting/894_loci/centroids_unsurveyed.csv", header=T))



In [ ]:
a=-10
b=10


x <- list(Distances=Distances, Assignments=Assignments, Sampled_reefs=Sampled_reefs, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions) #put inputs into a list because that's the bbmle format

Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))
Sim2013Fit

#doen't look right. Maybe fix theta, optimize k

In [ ]:
#monsoon season prep
NEM_months <- c(11, 12, 1, 2, 3, 4)
SWM_months <- c(5, 6, 7, 8, 9, 10)

NEM <- conn_mat_full4 %>%
    filter(month %in% NEM_months) %>%
    group_by(source, destination) %>%
    summarise(conn=max(fraction))

SWM <- conn_mat_full4 %>%
    filter(month %in% SWM_months) %>%
    group_by(source, destination) %>%
    summarise(conn=max(fraction))

__CELL BELOW IS WORKING FOR GLM OF OBSERVED DATA BY SIMULATED DATA__

In [ ]:
head(GenSimConn)

In [ ]:
sum(is.na(GenSimConn)==T) #should be 0

__run GLM observed genetic dispersal predicted by simulations__

In [ ]:
mean(GenSimConn$obs_disp) #should be 0.024
var(GenSimConn$obs_disp) #should be 0.054
#overdispersion?
#no, because residual deviance of the model is not greater than the df
#Residual deviance:  38.221  on 948  degrees of freedom

In [ ]:
#plan:
#fit 3 successive log-linear models starting with main efects only(fit1), main effects plus all 2-way interactions (fit2), and added in all 3 way interactions (fit3)


In [ ]:
NullMod <- glm(obs_disp ~ dist_km , data=GenSimConn, family="poisson")
summary(NullMod)

In [ ]:
ObsModExpYearFit1 <- glm(obs_disp ~ AnnRecPart + YearSampled + direction + dist_km +SourcePropSamp +DestPropSamp, data=GenSimConn, family="poisson")


In [ ]:
summary(ObsModExpYearFit1)

In [ ]:
#only main effects, first find worthwhile predictors with AIC
#the intercept is very significant but I think that just means 0s are more common in the data. totally true!

ObsModExpYearFit1 <- glm(obs_disp ~ AnnRecPart + YearSampled + direction + dist_km +SourcePropSamp +DestPropSamp, data=GenSimConn, family="poisson")
MainEff_AnnRecPart <- update(ObsModExpYearFit1, . ~ . -AnnRecPart)
MainEff_YearSampled <- update(ObsModExpYearFit1, . ~ . -YearSampled)
MainEff_direction <- update(ObsModExpYearFit1, . ~ . -direction)
MainEff_dist_km <- update(ObsModExpYearFit1, . ~ . -dist_km)
MainEff_SourcePropSamp <- update(ObsModExpYearFit1, . ~ . -SourcePropSamp)
MainEff_DestPropSamp <- update(ObsModExpYearFit1, . ~ . -DestPropSamp)
#drop the 3 terms AIC doesn't support? maybe not though, because when there's an interaction that's the best model
MainEff_ARP_DPrp_Dist <- glm(obs_disp ~ YearSampled + direction +SourcePropSamp, data=GenSimConn, family="poisson")
#add the interaction between year and simulations
ObsModExpYearFit2IntYear <- glm(obs_disp ~ AnnRecPart*YearSampled + direction+SourcePropSamp, data=GenSimConn, family="poisson")
ObsModExpYearFit2IntDir <- glm(obs_disp ~ AnnRecPart*YearSampled + direction+SourcePropSamp, data=GenSimConn, family="poisson")
ObsModExpYearFit2IntDirNoROMS <- glm(obs_disp ~ AnnRecPart+YearSampled*direction+SourcePropSamp, data=GenSimConn, family="poisson")
ObsModExpYearFit3Int <- glm(obs_disp ~ AnnRecPart*YearSampled*direction+SourcePropSamp, data=GenSimConn, family="poisson")

FullModNoROMSDestPropSamp <- glm(obs_disp ~  YearSampled + direction + dist_km +SourcePropSamp , data=GenSimConn, family="poisson")
FullModNoROMSDestPropSampInt <- glm(obs_disp ~  YearSampled* direction + dist_km +SourcePropSamp , data=GenSimConn, family="poisson")

MainEffAIC <- as.data.frame(AIC(NullMod, FullModNoROMSDestPropSampInt, FullModNoROMSDestPropSamp, ObsModExpYearFit1, MainEff_AnnRecPart, MainEff_YearSampled, MainEff_direction, MainEff_dist_km, MainEff_SourcePropSamp, MainEff_DestPropSamp, ObsModExpYearFit2IntDir, ObsModExpYearFit2IntYear, ObsModExpYearFit2IntDirNoROMS, ObsModExpYearFit3Int))
MainEffAIC$model <- row.names(MainEffAIC)
MainEffAIC <- MainEffAIC %>% arrange(AIC)
MainEffAIC
#/write.csv(MainEffAIC, file="~/oceanography/script_output/GLM/GLMObsDispAIC.csv",row.names=F,  quote=F)


In [ ]:
FullModNoROMSDestPropSampInt <- glm(obs_disp ~  YearSampled* direction+direction+YearSampled + dist_km +SourcePropSamp , data=GenSimConn, family="poisson")

summary(FullModNoROMSDestPropSampInt)

In [ ]:
AIC(test, MainEff_AnnRecPart,MainEff_DestPropSamp )

In [ ]:
summary(ObsModExpYearFit2Int ) #judging by the significant intercept (which should be AnnRecPart:Year2012?) when there's more particles expected from the ROMS model, there's actually less in the observed data.
ObsModExpYearFit2Int_Sum <- tidy(ObsModExpYearFit2Int)
#write.csv(ObsModExpYearFit2Int_Sum, file="~/oceanography/script_output/GLM/YearDispIntBestModSum.csv",row.names=F,  quote=F)


In [ ]:
ROMSFull <- glm(AnnRecPart ~ as.factor(SimYear) + direction + dist_km + source + destination, data=GenSimConn, family="poisson")
#MainEff_AnnRecPart <- update(ObsModExpYearFit1, . ~ . -AnnRecPart)

In [ ]:
summary(ROMSFull)

In [ ]:
summary(SimConnDFDir$NormSourceMonsoon)

In [ ]:
#what are the routes that create these peaks in direction annually?

#are there similar patterns seasonally?

#plot the connectivity matrix with of the roms model

In [ ]:
head(SimConnDFMeta %>% dplyr::select(AnnRecPart, direction, SimYear, source, destination))
SimConnDFMeta %>%

In [ ]:
hist(SimConnDFMeta$direction, breaks=100)

In [ ]:
summary(SimConnDFMeta$direction)
(SimConnDFMeta$direction)

In [ ]:
#this isn't significant, which means the residual difference is small enough here to say the model fits reasonably well
with(test_mod, cbind(res.deviance = deviance, df = df.residual,
  p = pchisq(deviance, df.residual, lower.tail=FALSE)))


In [ ]:
#how much devience is explained here?

In [ ]:
plot(obs_disp ~ ExpDisp, data=SimPlusGenWithDist)

In [ ]:
#plot(test_mod_int)

In [ ]:
#plot predicted model
range(SimPlusGenWithDist$ExpDisp)

In [ ]:
exp_disp_range <- seq(0, 66114, 1)

In [ ]:
obs_disp_pred <- predict(test_mod_int,type="response")

In [ ]:
plot( SimPlusGenWithDist$ExpDisp, SimPlusGenWithDist$obs_disp, pch = 16)#, xlab = "WEIGHT (g)", ylab = "VS")

lines(obs_disp_pred)